<a href="https://colab.research.google.com/github/atulpatildbz/smart-speedup-colab/blob/master/smart_speedup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

mount the drive

In [0]:
from google.colab import drive
drive.mount('/gdrive', )
%cd /gdrive

go to the directory

In [0]:
%cd /gdrive/'My Drive'
%ls

In [0]:
!ffmpeg -i DaredevilS01E05_burned.mp4_output.mp4 -vf scale=-1:720 output.mp4

extract sup file if any

In [0]:
!ffmpeg -i Dragon.Ball.Super.S04E11.1080p.Blu-Ray.10-Bit.Dual-Audio.TrueHD.x265-iAHD.mkv -map 0:3 -c:s copy "Dragon.Ball.Super.S04E11.1080p.Blu-Ray.10-Bit.Dual-Audio.TrueHD.x265-iAHD.sup"
!ffmpeg -i Dragon.Ball.Super.S04E12.1080p.Blu-Ray.10-Bit.Dual-Audio.TrueHD.x265-iAHD.mkv -map 0:3 -c:s copy "Dragon.Ball.Super.S04E12.1080p.Blu-Ray.10-Bit.Dual-Audio.TrueHD.x265-iAHD.sup"

In [0]:
!python sup2srt.py Dragon.Ball.Super.S04E11.1080p.Blu-Ray.10-Bit.Dual-Audio.TrueHD.x265-iAHD.sup
!python sup2srt.py Dragon.Ball.Super.S04E12.1080p.Blu-Ray.10-Bit.Dual-Audio.TrueHD.x265-iAHD.sup

install dependencies

In [0]:
!sudo apt install tesseract-ocr
!pip install pysrt
!pip install pytesseract

ffprobe

In [0]:
!ffprobe DaredevilS01E05_burned.mp4_output.mp4

In [0]:
!ffprobe -v error -select_streams v:0 -show_entries stream=width,height -of csv=s=x:p=0 S04E28.mkv_output.mp4

run code to extract srt from sup file

In [0]:
!python sup2srt.py 07.sup

code in case py file isn't there

In [0]:
import sys, pytesseract
from pgsreader import PGSReader
from imagemaker import make_image

from pysrt import SubRipFile, SubRipItem, SubRipTime

from tqdm import tqdm

supFile = '05.sup'
pgs = PGSReader(supFile)

srtFile = ".".join(supFile.split('.')[:-1])+".srt"

srt = SubRipFile()

# get all DisplaySets that contain an image
print("Loading DisplaySets...")
allsets = [ds for ds in tqdm(pgs.iter_displaysets())]

print(f"Running OCR on {len(allsets)} DisplaySets and building SRT file...")
subText = ""
subStart = 0
subIndex = 0

In [0]:
for ds in tqdm(allsets):
    try:
      if ds.has_image:
          # get Palette Display Segment
          pds = ds.pds[0]
          # get Object Display Segment
          ods = ds.ods[0]

          img = make_image(ods, pds)
          subText = pytesseract.image_to_string(img, lang='eng',  config='-l eng --oem 1 --psm 7')
          subStart = ods.presentation_timestamp
      else:
          startTime = SubRipTime(milliseconds=int(subStart))
          endTime = SubRipTime(milliseconds=int(ds.end[0].presentation_timestamp))
          srt.append(SubRipItem(subIndex, startTime, endTime, subText))
          subIndex += 1
    except:
      pass

In [0]:
print(f"Done. SRT file saved as {srtFile}")
srt.save(srtFile, encoding='utf-8')

burn original sup in the video. Change the streams according to output from ffprobe
https://trac.ffmpeg.org/wiki/HowToBurnSubtitlesIntoVideo

for single audio, remove 0 after a:

In [0]:
!ffmpeg -i 12.mkv -filter_complex "[0:v][0:s:0]overlay[v]" -map "[v]" -map 0:a:0 -vcodec libx264 -crf 27 -preset ultrafast -c:a copy 12_burned.mp4
!ffmpeg -i 13.mkv -filter_complex "[0:v][0:s:0]overlay[v]" -map "[v]" -map 0:a:0 -vcodec libx264 -crf 27 -preset ultrafast -c:a copy 13_burned.mp4

run the main file. use all the appropriate options based on weather we have converted and burnt sup file already.

In [0]:
!python splitspeedconcatV2.py -i 12_burned.mp4 -s 12.srt -ds 1.3 -ss 3.7 --use_slower_split
!python splitspeedconcatV2.py -i 13_burned.mp4 -s 13.srt -ds 1.3 -ss 3.7 --use_slower_split

master cell

In [0]:
import timeit
start_time = timeit.default_timer()
# !sudo apt install tesseract-ocr
# !pip install pysrt
# !pip install pytesseract
# !ffmpeg -i DaredevilS01E01.mkv -map 0:2 -c:s copy "DaredevilS01E01.sup"
# !ffmpeg -i DaredevilS01E01.mkv -filter_complex "[0:v][0:s:0]overlay[v]" -map "[v]" -map 0:a -vcodec libx264 -crf 27 -preset ultrafast -c:a copy DaredevilS01E01_burned.mp4
# !python sup2srt.py DaredevilS01E01.sup
!python splitspeedconcatV2.py -i DaredevilS01E01_burned.mp4 -s DaredevilS01E01.srt -ds 1.35 -ss 3.7 --use_slower_split
!ffmpeg -i DaredevilS01E01_burned.mp4_output.mp4 -preset ultrafast -c:v libx265 -crf 28 -c:a aac -b:a 128k DaredevilS01E01.mkv_output_compressed.mp4
elapsed = timeit.default_timer() - start_time
print(f"Completed! Whole process took {elapsed} amount of time")

In [0]:
!python splitspeedconcatV2.py -i Dragon.Ball.Super.S04E01.1080p.Blu-Ray.10-Bit.Dual-Audio.TrueHD.x265-iAHD.mkv -s Dragon.Ball.Super.S04E01.1080p.Blu-Ray.10-Bit.Dual-Audio.TrueHD.x265-iAHD.srt -ds 1.3 -ss 3.7 --use_slower_split
!python splitspeedconcatV2.py -i Dragon.Ball.Super.S04E06.1080p.Blu-Ray.10-Bit.Dual-Audio.TrueHD.x265-iAHD.mkv -s Dragon.Ball.Super.S04E06.1080p.Blu-Ray.10-Bit.Dual-Audio.TrueHD.x265-iAHD.srt -ds 1.3 -ss 3.7 --use_slower_split

mega download

In [0]:
!pip install mega.py
from mega import Mega
mega = Mega()
m = mega.login("atulpatildbz@gmail.com", "Dragonballgt@8")
m.get_files()

In [0]:
import timeit
start_time = timeit.default_timer()
file = m.find('DaredevilS01E05.mkv')
m.download(file)
elapsed = timeit.default_timer() - start_time
print(f"Completed! took {elapsed} amount of time")

compression

In [0]:
!ffmpeg -i S04E29.mkv_output.mp4 -preset ultrafast -c:v libx265 -crf 28 -c:a aac -b:a 128k S04E29.mkv_output_compressed.mp4
!ffmpeg -i S04E30.mkv_output.mp4 -preset ultrafast -c:v libx265 -crf 28 -c:a aac -b:a 128k S04E30.mkv_output_compressed.mp4